In [1]:
!pip install facenet-pytorch
!pip install mmcv
!pip install youtube-dl
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display
import itertools
import pandas as pd
import pickle
from pathlib import Path
import os
import copy
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Don't forget to mount gdrive
# from google.colab import drive
# drive.mount('/content/drive')

     |████████████████████████████████| 1.9 MB 5.0 MB/s 
     |████████████████████████████████| 390 kB 5.0 MB/s 
     |████████████████████████████████| 185 kB 60.1 MB/s 
  Created wheel for mmcv: filename=mmcv-1.3.17-py2.py3-none-any.whl size=573414 sha256=b438625a8cd8a476736d36ecbafca5edf46bf9334793d8b684fe4bf05c872733
  Stored in directory: /root/.cache/pip/wheels/4d/44/de/6831f4485611ea75714e7e8fad8d12b7f04933481c141f830b
Successfully built mmcv
     |████████████████████████████████| 1.9 MB 5.0 MB/s 


In [2]:
# Directories

# Video files
video_folder = "/content/drive/MyDrive/projects/pose_detection/videos"
# Reference images
ref_img_folder = "/content/drive/MyDrive/projects/pose_detection/data/reference_images"
# Reference list location
fr_reflist = "/content/drive/MyDrive/projects/pose_detection/data/fr_ref.csv"
# Final results folder on GDrive
fr_results_folder = "/content/drive/MyDrive/projects/pose_detection/results/face_recognition/"
# Final results folder on GDrive with similarity scores
fr_results_similarity_folder = "/content/drive/MyDrive/projects/pose_detection/results/face_recognition_similarity/"

In [29]:
# The code expects a dataframe with two columns
# video, which gives the name of the video
# ref_img, which gives the reference image of the candidate who is in the video
df = pd.read_csv(fr_reflist)
df

,video,ref_img,year
0,ATGEN_AL_BEDSOLE_BIGGER_THAN_ALL_OF_US.wmv,AL_ATTORNEY_GENERAL_BEDSOLE__CHESS.jpg,2018
1,ATGEN_AL_BEDSOLE_DON'T_DO_BUSINESS_WITH_CROOKS...,AL_ATTORNEY_GENERAL_BEDSOLE__CHESS.jpg,2018
2,ATGEN_AL_BEDSOLE_TRUMP_REPUBLICAN.wmv,AL_ATTORNEY_GENERAL_BEDSOLE__CHESS.jpg,2018
3,ATGEN_AL_CHRISTIE_VOTE_JUSTICE_5.wmv,AL_ATTORNEY_GENERAL_CHRISTIE__CHRIS.jpg,2018
4,ATGEN_AL_CHRISTIE_WALK_HUMBLY.wmv,AL_ATTORNEY_GENERAL_CHRISTIE__CHRIS.jpg,2018
...,...,...,...
7622,USSEN_WV_CAPITO_CONNECT.wmv,WV_US_SENATE_CAPITO__SHELLEY_MOORE.jpg,2020
7623,USSEN_WV_CAPITO_SPECIAL.wmv,WV_US_SENATE_CAPITO__SHELLEY_MOORE.jpg,2020
7624,USSEN_WV_CAPITO_THE_BOLD_ACT.wmv,WV_US_SENATE_CAPITO__SHELLEY_MOORE.jpg,2020
7625,USSEN_WV_CAPITO_TRUSTED_LEADER.wmv,WV_US_SENATE_CAPITO__SHELLEY_MOORE.jpg,2020


In [30]:
# -- Remove this again later --
df = df.sort_values('year', ascending = False)
df = df.reset_index(drop = True)
df

,video,ref_img,year
0,USSEN_WV_SWEARENGIN_BROKEN_60.wmv,WV_US_SENATE_SWEARENGIN__PAULA_JEAN.jpg,2020
1,HOUSE_NY23_REED_BRICK.wmv,NY_US_HOUSE_REED__TOM.jpg,2020
2,HOUSE_NY22_BRINDISI_SHERRILL_MANUFACTURING.wmv,NY_US_HOUSE_BRINDISI__ANTHONY.jpg,2020
3,HOUSE_NY22_BRINDISI_REPUBLICANS.wmv,NY_US_HOUSE_BRINDISI__ANTHONY.jpg,2020
4,HOUSE_NY22_BRINDISI_OUR_POLITICS_IS_BROKEN.wmv,NY_US_HOUSE_BRINDISI__ANTHONY.jpg,2020
...,...,...,...
7622,HOUSE_VA10_COMSTOCK_THE_FIRST_ONE_60.wmv,VA_US_HOUSE_COMSTOCK__BARBARA.jpg,2018
7623,HOUSE_VA10_COMSTOCK_THE_FIRST_ONE.wmv,VA_US_HOUSE_COMSTOCK__BARBARA.jpg,2018
7624,HOUSE_VA10_COMSTOCK_NRCC_THE_FIRST_ONE.wmv,VA_US_HOUSE_COMSTOCK__BARBARA.jpg,2018
7625,HOUSE_VA09_GRIFFITH_NEW_TAX_CUTS.wmv,VA_US_HOUSE_GRIFFITH__MORGAN.jpg,2018


In [31]:
video_paths = video_folder + "/" + df.year.map(str) + "/" + df.video
ref_img_paths = ref_img_folder + "/" + df.year.map(str) + "/" + df.ref_img

df['video_paths'] = video_paths
df['ref_img_paths'] = ref_img_paths

In [34]:
#instantiate the MTCNN which does face detection
#set keep_all to True so that all faces in each frame are recorded, rather than just one
#min_face_size to 5 (default is 20) so that smaller faces can also be detected
mtcnn = MTCNN(keep_all=True, device=device, min_face_size=5)
#Instantiate the InceptionResnet which does face recognition
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [35]:
for v in range(len(df)):

  current_video_location =  df.video_paths[v]
  current_refimg_location =  df.ref_img_paths[v]
  current_video_name = Path(current_video_location).stem
  candidate_boxes_dict_location = fr_results_folder + str(df.year[v]) + "_" + current_video_name + '_cand_boxes.pkl'
  candidate_boxes_similarity_dict_location = fr_results_similarity_folder + str(df.year[v]) + "_" + current_video_name + '_cand_boxes.pkl'

  current_video_exists = os.path.exists(current_video_location)
  current_refimg_exists = os.path.exists(current_refimg_location)  
  output_not_exists = not os.path.exists(candidate_boxes_dict_location)

  if (current_video_exists & current_refimg_exists & output_not_exists):

    # Read in the video
    video = mmcv.VideoReader(current_video_location)
    # Extract invididual frames from the video
    # Each of these are PIL images that can be displayed in ipython, i.e. frames[0]
    frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

    # Detect faces
    frames_tracked = []
    aligned = []
    boxes_l = []
    for i, frame in enumerate(frames):
        
        # Detect faces
        # First object are all the boxes
        # Second object are the probabilities of the detected faces
        batch_boxes, batch_probs = mtcnn.detect(frame)
        # Use the boxes to get the cropped-out images as tensors
        # This is different from what the package expects
        # Normally, it either gives you only the cropped images, or only the boxes
        # But the extract function is what it uses internally to go from the boxes
        #  to the cropped-out images
        x_aligned = mtcnn.extract(frame, batch_boxes, None)

        # If at least one face has been detected, x_aligned will not be none
        if x_aligned is not None:
          # How many faces have been detected?
          detected_faces = x_aligned.shape[0]
          # Loop over each detected face
          for j in range(detected_faces):
            # If there are more than one, change dimensions from
            # [3,3,160,160] to [3,160,160]
            if detected_faces>1:
              face_i = torch.unbind(x_aligned)[j]
            # If there is one, change dimensions from
            # [1,3,160,160] to [3,160,160]
            else:
              face_i = x_aligned.squeeze(0)
            # Either way, add the face (its tensor) we just processed to the list of faces
            aligned.append(face_i)
            # Also add the number of the frame and the number of the face within the frame
            # to a seperate list
            boxes_l.append([i,j,batch_boxes[j]])

    print('\nFinished video embeddings.')

    # Calculate embeddings
    # The embeddings could all be calculated at the same time in theory
    # In practice, there is not enough RAM
    # So we chunk them
    # Calculate the number of chunks if we want to do chunk size 50
    chunks = (len(aligned) - 1) // 50 + 1
    embeddings_batch = []
    for i in range(chunks):
        # Chunk the list of image tensors
        batch = aligned[i*50:(i+1)*50]
        # Stack them as a tensor with a new dimension
        aligned_s = torch.stack(batch).to(device)
        # Process them all
        # Put the embeddings on the cpu
        embeddings = resnet(aligned_s).detach().cpu()
        # Append them to the list of embeddings
        embeddings_batch.append(embeddings)

    # Un-chunk the embeddings so that we get a flat list where each item is one face
    embeddings_l = list(itertools.chain(*embeddings_batch))

    # Compare the embeddings of the video to a reference embedding for the candidate
    # Read in the ballotpedia image
    candidate_img = Image.open(current_refimg_location)  
    # Some of these images have an unused alpha channel which messes with pytorch
    # remove it
    if candidate_img.mode == 'RGBA':
      candidate_img = candidate_img.convert('RGB')
    # Detect and post-process the face
    candidate_aligned_all = mtcnn(candidate_img)
    # If the fr algorithm doesnt find any face, skip
    if candidate_aligned_all is not None:
      candidate_aligned = candidate_aligned_all[0]
      #'Stack' the one face, this is because the resnet expects that number of dimensions
      candidate_aligned_stacked = torch.stack([candidate_aligned]).to(device)
      # Get the embeddings for the candidate's reference image
      candidate_embeddings = resnet(candidate_aligned_stacked).detach().cpu()

      # Compare the embeddings of the ballotpedia image
      # to the embeddings of every detected face in the video
      dists = [(e1 - candidate_embeddings).norm().item() for e1 in embeddings_l]

      # Which of the distances are smaller than 1, meaning that they are the candidate?
      candidate_face_indices = np.where(np.array(dists)<1)
      # Put only those in a new list
      candidate_boxes = [boxes_l[i] for i in candidate_face_indices[0].tolist()]
      # Make a dictionary where the key is the frame number and the item is the bounding box
      candidate_boxes_dict = {i:k for i, j, k in candidate_boxes}

      # Save the results
      with open(candidate_boxes_dict_location,'wb') as f:
        pickle.dump(candidate_boxes_dict, f)

      # Make a copy of the results and add the similarity scores
      boxes_l2 = copy.deepcopy(boxes_l)
      for i,j in enumerate(boxes_l2):
        j.append(dists[i])

      # Save the results with the similarity scores
      with open(candidate_boxes_similarity_dict_location,'wb') as f:
        pickle.dump(boxes_l2, f)

      print('\rFinished processing video: {}'.format(v), end='')


Finished video embeddings.
Finished processing video: 397
Finished video embeddings.
Finished processing video: 399
Finished video embeddings.
Finished processing video: 602
Finished video embeddings.

Finished video embeddings.

Finished video embeddings.

Finished video embeddings.
Finished processing video: 1217
Finished video embeddings.
Finished processing video: 1218
Finished video embeddings.
Finished processing video: 1219
Finished video embeddings.
Finished processing video: 1220
Finished video embeddings.
Finished processing video: 1222
Finished video embeddings.
Finished processing video: 1223
Finished video embeddings.
Finished processing video: 1224
Finished video embeddings.
Finished processing video: 1225
Finished video embeddings.
Finished processing video: 1226
Finished video embeddings.
Finished processing video: 1227
Finished video embeddings.
Finished processing video: 1228
Finished video embeddings.
Finished processing video: 1229
Finished video embeddings.
Finish